Generate Dataset for all 5-stage terrain

In [1]:
import numpy as np
import cv2
from PIL import Image
import torch
import os
import torch
import torch.nn as nn
import torchvision.transforms.functional as F
import torch.nn.parallel as parallel

In [2]:
def load_image(file_path):
    return Image.open(file_path)

def save_image(image_array, file_name):
    image = Image.fromarray(image_array)
    image.save(file_name)

# For Manual-CL dataset
def generate_dataset(image_paths, train_output_dir, test_output_dir, num_train_samples, num_test_samples):
    if not os.path.exists(train_output_dir):
        os.makedirs(train_output_dir)
    if not os.path.exists(test_output_dir):
        os.makedirs(test_output_dir)
        
    train_counter = 1
    test_counter = 1
    
    for image_path in image_paths:
        image = load_image(image_path)
        bitmap_array = np.array(image)

        # Generate training samples
        for i in range(num_train_samples):
            cropped_image = generate_cropped_image(bitmap_array)
            save_image(cropped_image, os.path.join(train_output_dir, f'{train_counter}.bmp'))
            train_counter += 1

        # Generate testing samples
        for i in range(num_test_samples):
            cropped_image = generate_cropped_image(bitmap_array)
            save_image(cropped_image, os.path.join(test_output_dir, f'{test_counter}.bmp'))
            test_counter += 1
            
def generate_cl(input_dir, output_dir, crops_per_image):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_counter = 1
    
    # Get sorted list of files
    filenames = sorted([f for f in os.listdir(input_dir) if f.startswith('level_') and f.endswith('.bmp')],
                       key=lambda x: int(x.split('_')[1].split('.')[0]))
    
    for filename in filenames:
        image_path = os.path.join(input_dir, filename)
        image = load_image(image_path)
        bitmap_array = np.array(image)
        
        for i in range(crops_per_image):
            cropped_image = generate_cropped_image(bitmap_array)
            save_image(cropped_image, os.path.join(output_dir, f'{image_counter}.bmp'))
            image_counter += 1
                
                
def generate_cropped_image(bitmap_array):
    pos_bmp_x, pos_bmp_y = np.random.uniform(32, 118, 2)
    vehicle_heading_global = np.random.uniform(0, 2*np.pi)
    
    pos_bmp_x = int(pos_bmp_x)
    pos_bmp_y = int(pos_bmp_y)
    
    center_x = bitmap_array.shape[0] // 2
    center_y = bitmap_array.shape[1] // 2
    shift_x = center_x - pos_bmp_x
    shift_y = center_y - pos_bmp_y

    shifted_map = np.roll(bitmap_array, shift_x, axis=0)
    shifted_map = np.roll(shifted_map, shift_y, axis=1)
    shifted_map = np.expand_dims(shifted_map, axis=0)
    r_map = torch.tensor(shifted_map)

    angle = np.degrees(np.pi + vehicle_heading_global) if np.degrees(vehicle_heading_global) < 0 else np.degrees(-np.pi + vehicle_heading_global)
    rotated_map = np.array((F.rotate(r_map, angle)).squeeze().cpu(), dtype=np.uint8)

    half_size_x = 64 // 2
    half_size_y = 64 // 2

    start_y = center_y - half_size_y
    end_y = center_y + half_size_y
    start_x = center_x - half_size_x
    end_x = center_x + half_size_x

    start_y = max(start_y, 0)
    end_y = min(end_y, rotated_map.shape[0])
    start_x = max(start_x, 0)
    end_x = min(end_x, rotated_map.shape[1])

    if end_y - start_y < 64:
        if start_y == 0:
            end_y = min(start_y + 64, rotated_map.shape[0])
        elif end_y == rotated_map.shape[0]:
            start_y = max(end_y - 64, 0)

    if end_x - start_x < 64:
        if start_x == 0:
            end_x = min(start_x + 64, rotated_map.shape[1])
        elif end_x == rotated_map.shape[1]:
            start_x = max(end_x - 64, 0)

    sub_array = rotated_map[start_x:end_x, start_y:end_y]

    return sub_array

# # Paths to the original images for Manual-CL dataset
# image_paths = [
#     '../data/terrain_bitmaps/Manual-CL/stage_1.bmp',
#     '../data/terrain_bitmaps/Manual-CL/stage_2.bmp',
#     '../data/terrain_bitmaps/Manual-CL/stage_3.bmp',
#     '../data/terrain_bitmaps/Manual-CL/stage_4.bmp',
#     '../data/terrain_bitmaps/Manual-CL/stage_5.bmp'
# ]

# # Generate Manual-CL dataset
# generate_dataset(image_paths, '../data/terrain_bitmaps/Manual-CL/TrainSet', '../data/terrain_bitmaps/Manual-CL/TestSet', 10000, 100)

# Generate Automatic-CL dataset
input_dir = '../data/terrain_bitmaps/Automatic-CL/TrainLevels'
output_dir = '../data/terrain_bitmaps/Automatic-CL/TrainSet-test'

generate_cl(input_dir, output_dir, crops_per_image=5)